In [ ]:
from configparser import ConfigParser

config = ConfigParser()
config.read('config.ini')
config['augmentations']['pad_trunc_noise_multiplier']


In [1]:
import ml.machineLearning as ml

ml.selectModel()[2]

/home/jianquan/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[0] saved_model/CNNtest_epoch0.pt


Using cache found in /home/jianquan/.cache/torch/hub/pytorch_vision_v0.10.0


'0'

In [ ]:
import torch, torchaudio
import IPython.display as ipd

trimmed_audio, b = torchaudio.load(
    'E:/asdas.wav')

trimmed_audio = trimmed_audio[0][0:1000000]
print(trimmed_audio.shape)
trimmed_audio = torchaudio.functional.vad(trimmed_audio, b)
print(trimmed_audio.shape)
ipd.Audio(trimmed_audio, rate=b)

In [ ]:
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
from tqdm import tqdm

import torch
import torchaudio
import random
import utils
from torch.utils.data import Dataset, DataLoader

from pathlib import Path
import os

class AudioDataset(Dataset):
    def __init__(self, audio_paths, device):
        self.audio_paths = audio_paths
        self.device = device

    def __len__(self):
        return len(self.audio_paths)

    def __getitem__(self, idx):
        waveform, sample_rate = torchaudio.load(self.audio_paths[idx])
        waveform = waveform[0].to(self.device)
        pad_len = 240000 - len(waveform)
        zero_pad = torch.zeros(pad_len, device=self.device)
        waveform = torch.concat((waveform, zero_pad),0)
        return waveform

audioPaths = list(Path('E:/Singapore Speech Corpus/Part 2.1').glob('**/*.wav'))
dataset = AudioDataset(audioPaths, torch.device(
    "cuda" if torch.cuda.is_available() else "cpu"))
training_loader = torch.utils.data.DataLoader(dataset, batch_size=1024, shuffle=False, num_workers=0)



class MyPipeline(torch.nn.Module):
    def __init__(
        self,
        input_freq=16000,
        resample_freq=8000
    ):
        super().__init__()
        self.resample = torchaudio.transforms.Resample(orig_freq=input_freq, new_freq=resample_freq)

    def forward(self, waveform: torch.Tensor) -> torch.Tensor:
        # Resample the input
        processed = self.resample(waveform)
        processed = torch.split(processed, 4*8000, dim=1)
        return processed


pipeline = MyPipeline()

pipeline.to(device=torch.device("cuda"))

for batch in tqdm(training_loader):
    pipeline(batch)


In [ ]:
a = torch.arange(16000).reshape(16,1000)
torch.split(a, 100, dim=1)

In [ ]:
import pyaudio
import wave
# from torchaudio.io import StreamReader

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 8000
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    print(data)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()


In [ ]:
import loader.AudioDataset as Augmentation
import torch
from loader.AudioDataset import createDataset
from torch.utils.data import Dataset
import audiomentations
import loader.utils as utils

env_paths = utils.getAudioPaths('E:/Processed Audio/ENV')

speech_paths = utils.getAudioPaths('E:/Processed Audio/SPEECH')

audio_train_dataset = createDataset(env_paths, speech_paths, transformParams = utils.getTransforms(True))

test_dataloader = torch.utils.data.DataLoader(
    audio_train_dataset,
    batch_size=64,
    num_workers=0,
    shuffle=True,
)

batch = next(iter(test_dataloader))
batch

In [3]:
import loader.utils as utils

train, val = utils.getAudioPaths('E:/Processed Audio', percent=0.9)

In [4]:
print(len(train[0]), len(train[1]))
print(len(val[0]), len(val[1]))


873 873
98 98


In [23]:
import torchaudio, torch
import loader.AudioDataset as AudioDataset
import scipy
import numpy as np
import matplotlib.pyplot as plt

a = [0,1,2,3,4,5]
print(a[:3],a[3:])

[0, 1, 2] [3, 4, 5]
